# Higher-order elasticity

* Author: Mauricio Fernández
* Last update: 2020-06-14

Table of contents

* [1. Example for C4](#1.-Example-for-C4)
* [2. Example for C6](#2.-Example-for-C6)

## Description
The present notebook corresponds to the higher-order elasticity example for aluminum, see section 3.2 in published manuscript. In higher-order elasticity, stiffness tensors of not only fourth-order, but also sixth-order might be considered. Orientation averages of such tensors in polycrystalline materials require adequate treatment. The present notebook demonstrates how the results for central ODFs can be applied to such cases.

In [1]:
import sympy as sym
import numpy as np
import datetime
now = datetime.datetime.now

import src.CentralODFAverage as cen
import src.TensorCalculusSympy as ts
import src.TensorCalculusNumpy as tn

## 1. Example for C4

The following example corresponds to the example documented in section 3.2, see equation (82).

Generate symbolic fourth-order tensor $\tilde{\mathbb{C}}_{\langle r \rangle}$ for $r=4$.

In [2]:
C = ts.gent('c',4)
len(C.free_symbols)

81

Solve minor and major index symmetry conditions and rewrite $\tilde{\mathbb{C}}_{\langle r \rangle}$ accordingly.

In [3]:
index_sym = [(1,0,2,3),(2,3,0,1)]
for isym in index_sym:
    C = ts.symmetrizeex(C,isym)
    print(C == sym.permutedims(C, isym))

True
True


Current number of free components in $\tilde{\mathbb{C}}_{\langle r \rangle}$ (classical results for triclinic linear hyperelasticity).

In [4]:
len(C.free_symbols)

21

Solve group conditions for cubic group (tetrahedral group $O_h$ with 24 rotations).

In [5]:
C = ts.symmetrize_group(C, ts.sg_cub)
print(C.free_symbols)
C

{c2222, c2121, c2211}


[[[[c2222, 0, 0], [0, c2211, 0], [0, 0, c2211]], [[0, c2121, 0], [c2121, 0, 0], [0, 0, 0]], [[0, 0, c2121], [0, 0, 0], [c2121, 0, 0]]], [[[0, c2121, 0], [c2121, 0, 0], [0, 0, 0]], [[c2211, 0, 0], [0, c2222, 0], [0, 0, c2211]], [[0, 0, 0], [0, 0, c2121], [0, c2121, 0]]], [[[0, 0, c2121], [0, 0, 0], [c2121, 0, 0]], [[0, 0, 0], [0, 0, c2121], [0, c2121, 0]], [[c2211, 0, 0], [0, c2211, 0], [0, 0, c2222]]]]

Insert material data of aluminum.

In [6]:
mat_data = {
    C[0, 0, 0, 0]: 108, C[1, 2, 1, 2]: 33, C[0, 0, 1, 1]: 59
}
C = C.subs(mat_data)
C

[[[[108, 0, 0], [0, 59, 0], [0, 0, 59]], [[0, 33, 0], [33, 0, 0], [0, 0, 0]], [[0, 0, 33], [0, 0, 0], [33, 0, 0]]], [[[0, 33, 0], [33, 0, 0], [0, 0, 0]], [[59, 0, 0], [0, 108, 0], [0, 0, 59]], [[0, 0, 0], [0, 0, 33], [0, 33, 0]]], [[[0, 0, 33], [0, 0, 0], [33, 0, 0]], [[0, 0, 0], [0, 0, 33], [0, 33, 0]], [[59, 0, 0], [0, 59, 0], [0, 0, 108]]]]

Transform to np array.

In [7]:
C = np.array(sym.flatten(C)).astype(np.float).reshape(4*[3,])
C

array([[[[108.,   0.,   0.],
         [  0.,  59.,   0.],
         [  0.,   0.,  59.]],

        [[  0.,  33.,   0.],
         [ 33.,   0.,   0.],
         [  0.,   0.,   0.]],

        [[  0.,   0.,  33.],
         [  0.,   0.,   0.],
         [ 33.,   0.,   0.]]],


       [[[  0.,  33.,   0.],
         [ 33.,   0.,   0.],
         [  0.,   0.,   0.]],

        [[ 59.,   0.,   0.],
         [  0., 108.,   0.],
         [  0.,   0.,  59.]],

        [[  0.,   0.,   0.],
         [  0.,   0.,  33.],
         [  0.,  33.,   0.]]],


       [[[  0.,   0.,  33.],
         [  0.,   0.,   0.],
         [ 33.,   0.,   0.]],

        [[  0.,   0.,   0.],
         [  0.,   0.,  33.],
         [  0.,  33.,   0.]],

        [[ 59.,   0.,   0.],
         [  0.,  59.,   0.],
         [  0.,   0., 108.]]]])

Load corresponding $\mathbb{D}_{\langle 2r \rangle \alpha}$.

In [8]:
r = C.ndim
Ds = cen.Dv_load(r, as_tensors=True)
Ds.shape

(5, 3, 3, 3, 3, 3, 3, 3, 3)

Compute $\|\mathbb{D}_{\langle 2r \rangle \alpha}[\tilde{\mathbb{C}}_{\langle r \rangle}]\|$, see equation (82) in published manuscript.

In [9]:
for i in range(r + 1):
    print('\t%2.4f' % tn.nf(tn.lm(Ds[i], C)))

	261.9145
	0.0000
	0.0000
	0.0000
	18.6226


## 2. Example for C6

The following example corresponds to the example documented in section 3.2, see equation (82).

Generate symbolic fourth-order tensor $\tilde{\mathbb{C}}_{\langle r \rangle}$ for $r=6$.

In [10]:
C = ts.gent('c', 6)
print(len(C.free_symbols))

729


Solve for corresponding index symmetries (up to 15 min, get some coffee :D).

In [11]:
t1 = now()
index_sym = [
    (1, 0, 2, 3, 4, 5),
    (2, 3, 0, 1, 4, 5),
    (0, 1, 4, 5, 2, 3)
]
for isym in index_sym:
    C = ts.symmetrizeex(C, isym)
t2 = now()
print(t2-t1)
print(len(C.free_symbols))

0:12:35.501190
56


Solve group conditions for cubic symmetry.

In [12]:
t1 = now()
C = ts.symmetrize_group(C, ts.sg_cub)
t2 = now()
print(t2-t1)
print(len(C.free_symbols))

0:03:08.415924
6


Insert material data for aluminum.

In [13]:
mat_data = {
    C[0, 0, 0, 0, 0, 0]: -1100,
    C[0, 0, 0, 0, 1, 1]: -371,
    C[0, 0, 1, 1, 2, 2]: 104,
    C[0, 0, 1, 2, 1, 2]: 39,
    C[0, 0, 0, 2, 0, 2]: -421,
    C[1, 2, 0, 2, 0, 1]: -22
}
C = C.subs(mat_data)
C = np.array(sym.flatten(C)).astype(np.float).reshape(6*[3,])

Load corresponding $\mathbb{D}_{\langle 2r \rangle \alpha}$.

In [14]:
t1 = now()
r = C.ndim
Ds = cen.Dv_load(r, as_tensors=True)
t2 = now()
print(t2-t1)

0:00:14.504840


Compute norms, see equation (82) in published manuscript.

In [15]:
for i in range(r + 1):
    print('\t%2.4f' % tn.nf(tn.lm(Ds[i], C)))

	4033.1707
	0.0000
	0.0000
	0.0000
	694.5125
	0.0000
	1504.3138
